<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Поиск-реплик-приветствия-менеджера" data-toc-modified-id="Поиск-реплик-приветствия-менеджера-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Поиск реплик приветствия менеджера</a></span></li><li><span><a href="#Извлечение-имени-менеджера" data-toc-modified-id="Извлечение-имени-менеджера-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Извлечение имени менеджера</a></span></li><li><span><a href="#Извлечение-названия-компании" data-toc-modified-id="Извлечение-названия-компании-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Извлечение названия компании</a></span></li><li><span><a href="#Поиск-реплик-прощания-менеджера" data-toc-modified-id="Поиск-реплик-прощания-менеджера-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Поиск реплик прощания менеджера</a></span></li><li><span><a href="#Проверка-менеджера" data-toc-modified-id="Проверка-менеджера-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка менеджера</a></span></li><li><span><a href="#Экспорт-таблицы" data-toc-modified-id="Экспорт-таблицы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Экспорт таблицы</a></span></li></ul></div>

In [1]:
import pandas as pd
import spacy
import translators as ts

from natasha import Segmenter, NewsEmbedding, NewsNERTagger, Doc

import nltk
import pymorphy2

import numpy as np

Using state Tyumenâ Oblast server backend.


In [2]:
spacy.prefer_gpu()

True

In [3]:
data = pd.read_csv('test_data.csv')
data

,dlg_id,line_n,role,text
0,0,0,manager,Алло
1,0,1,client,Алло здравствуйте
2,0,2,manager,Добрый день
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,manager,Ага
...,...,...,...,...
475,5,138,client,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,client,Все я вам высылаю счет и с вами на связи если ...
477,5,140,manager,Спасибо спасибо
478,5,141,manager,Да да тогда созвонимся ага спасибо вам давайте


## Поиск реплик приветствия менеджера

Поскольку приветствие менеджера должно быть формальным, а темы для разговора весьма ограничены, для определение реплик с приветствием будет достаточно самого простого способа.

Кортеж с корнями формальных слов-приветствий:

In [4]:
greets = ('здравст', 'привет', 'здоров', 'здрави', 'добры', 'доброго')

Функция для поиска приветствия:

In [5]:
def Find_Greeting(x, limit=3):
    
    if x['line_n'] >= limit:
        return np.nan
    
    text = x['text'].lower()
    
    for greet in greets:
        if greet in text:
            return True

In [6]:
data['greeting'] = data[data['role'] == 'manager'].apply(Find_Greeting, axis = 1)

In [7]:
data[data['greeting'].notna()]

,dlg_id,line_n,role,text,greeting
2,0,2,manager,Добрый день,True
109,1,0,manager,Да здравствуйте когда заканчивается,True
165,2,1,manager,Здравствуйте,True
249,3,0,manager,Добрый день,True
304,4,2,manager,Добрый,True


## Извлечение имени менеджера

Реплику можно будет определить по начилию имени в столбце. Представление себя может быть осуществлено только в начальной форме имени и начале диалога.

In [8]:
spacy_ru = spacy.load('ru_core_news_lg')
morph = pymorphy2.MorphAnalyzer()

prob_thresh = 0.4

In [9]:
def Extract_Name(x, limit=3):
    
    if x['line_n'] >= limit:
        return np.nan
    
    # поиск имени с помощью spacy
    for ent in spacy_ru(x['text']).ents:
        if ent.label_ == 'PER' and ent.text.lower() == ent.lemma_:
            #print('spacy')
            return ent.text
        
    # поиск имени с помощью pymorphy2    
    name = ''
    for word in nltk.word_tokenize(x['text']):
        for p in morph.parse(word):
            if 'Name' in p.tag and p.score >= prob_thresh and word == p.normal_form:
                #print('morph')
                if name == '':
                    name += p.normal_form
                else:
                    name = name + ' ' + p.normal_form
    
    if name == '':
        return np.nan
    else:
        return name

In [10]:
data['name_manager'] = data[data['role'] == 'manager'].apply(Extract_Name, axis = 1)

In [11]:
data['name_manager'].notna().sum()

0

Похоже, что менеджер ни разу не представился.

## Извлечение названия компании

Извлечение с помощью spacy и Natasha. В трудных случаях с помощью перевода на английский язык.

In [12]:
spacy_eng = spacy.load('en_core_web_lg')

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
segmenter = Segmenter()

Кортеж для использользования с переводом:

In [13]:
orgs = ('орган', 'фирм', 'предприятие', 'завод', 'ооо', 'оао', 'обществ', 'заведен', 'компан')

In [14]:
ts_works = True
def Extract_Org(x):
    
    fakes = ['амо']   
    global ts_works
    orgname = ''
    
    if len(x) < 12:
        return np.nan
    
    # spacy русском
    for ent in spacy_ru(x).ents:
        if ent.label_ == 'ORG':
            if orgname == '':
                orgname = ent.text
            else:
                orgname = orgname + ', ' + ent.text
                
    if orgname != '' and not any(elem in fakes for elem in orgname.split()):
        return orgname
        
    # Natasha
    doc = Doc(x)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        if span.type == 'ORG':
            if orgname == '':
                    orgname = span.text
            else:
                orgname = orgname + ', ' + span.text
                
    if orgname != '' and not any(elem in fakes for elem in orgname.split()):
        return orgname
    
    # spacy с переводом на английский
    if ts_works:
         for org in orgs:
                if org in x.lower():
                    try:   
                        tstext = ts.google(x)
                        #print(x)
                    except:
                        print('Перевод не удался.')
                        ts_works = False
                        break
                        
                    for ent in spacy_eng(tstext).ents:
                        if ent.label_ == 'ORG':
                            if orgname == '':
                                orgname = ent.text
                            else:
                                orgname = orgname + ', ' + ent.text
    if orgname != '' and not any(elem in fakes for elem in orgname.split()):
        return orgname

In [15]:
%%time
data['name_org'] = data['text'].apply(Extract_Org)

CPU times: total: 12.8 s
Wall time: 31.7 s


In [16]:
data['name_org'].notna().sum()

6

In [17]:
data[data['name_org'].notna()]

,dlg_id,line_n,role,text,greeting,name_manager,name_org
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,NaN,NaN,Dijital Business
111,1,2,client,Меня зовут ангелина компания диджитал бизнес з...,NaN,NaN,Dijital
167,2,3,client,Меня зовут ангелина компания диджитал бизнес з...,NaN,NaN,Angelina Company Dijital
177,2,13,client,Диджитал бизнес,NaN,NaN,Диджитал бизнес
251,3,2,client,Добрый меня максим зовут компания китобизнес у...,NaN,NaN,"Maxim, Kitobovusiness"
272,3,23,manager,Транспортная компания,NaN,NaN,Транспортная компания


## Поиск реплик прощания менеджера

Прощание также должно было формальным.

In [18]:
goodbyes = ('свидан','проща','скорог','скорой','встреч','давай','пиши','звони','всего наилучшего','до завтра','до послезавтра')

In [19]:
max_lines_dict = data[data['role'] == 'manager'].groupby('dlg_id')['line_n'].max().to_dict()

In [20]:
def Find_Bye(x, limit=3):
    
    if x['line_n'] <= max_lines_dict[x['dlg_id']] - limit:
        return np.nan
    
    text = x['text'].lower()
    
    for bye in goodbyes:
        if bye in text:
            return True

In [21]:
data['goodbye'] = data[data['role'] == 'manager'].apply(Find_Bye, axis = 1)
data[data['goodbye'].notna()]

,dlg_id,line_n,role,text,greeting,name_manager,name_org,goodbye
301,3,52,manager,Да до свидания,NaN,NaN,None,True
336,4,34,manager,Угу да до свидания,NaN,NaN,None,True
478,5,141,manager,Да да тогда созвонимся ага спасибо вам давайте,NaN,NaN,None,True


## Проверка менеджера

«В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [22]:
data[data['role'] == 'manager'].groupby('dlg_id')[['greeting','goodbye']].sum().apply(lambda x: x['greeting'] is True and x['goodbye'] is True, axis = 1)

dlg_id
0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

Менеджер поздоровался и попрощался только в диалогах 3 и 4.

## Экспорт таблицы

In [23]:
data.to_csv('test_data.csv')